### 전처리/증강 데이터셋 버전 
#### 형식
- v{대분류}_{인덱스}
#### 대분류
- v1: 전처리만 적용
- v2: v1 + swap만 적용 
- v3: v2 + 데이터 증강 기법 적용
- v4: v3 + 매우 실험적인 전처리/증강 기법 적용
#### 인덱스
- 01부터 99까지 차례로 순서를 매김
#### 예시
- 맞춤법 교정 데이터셋 -> v1_01
- 특수문자 제거 데이터셋 -> v1_02
- 특수문자 제거, 띄어쓰기 교정, swap 데이터셋 -> v2_01
- 맞춤법 교정, swap, RS, SR -> v3_01
- ChatGPT api 증강 데이터셋 -> v4_01

In [1]:
import pandas as pd
import warnings

# 모든 경고 메시지 무시
warnings.filterwarnings("ignore")

In [2]:
def print_label_count(df):
    """라벨 분포 확인용"""
    print(len(df))
    df["label_int"] = pd.cut(
        df["label"],
        bins=[x for x in range(6)],
        labels=[x for x in range(5)],
        right=False,
    )
    print(df.groupby("label_int")["id"].count())

In [3]:
def replace_special_letters(df):
    """특수문자 제거 \n 
    초성(ㄱ-ㅎ), 중성(ㅏ-ㅣ), 완성된 한글(가-힣), 알파벳(A-Za-z), 숫자(0-9), 그리고 공백(\s)만 허용"""
    df["sentence_1"] = df["sentence_1"].str.replace(
        r"[^A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ\s]", "", regex=True
    )
    df["sentence_2"] = df["sentence_2"].str.replace(
        r"[^A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ\s]", "", regex=True
    )
    return df

In [4]:
from hanspell import passportKey, spell_checker

# passportKey 설정
passportKey.init()


def spell_check(df):
    """맞춤법(오탈자, 띄어쓰기 등 전부) 교정"""
    df["sentence_1"] = df.apply(lambda row: spell_checker.check(row["sentence_1"]).checked, axis=1)
    df["sentence_2"] = df.apply(lambda row: spell_checker.check(row["sentence_2"]).checked, axis=1)
    return df

In [5]:
import pykospacing


def spacing_text(df):
    """띄어쓰기만 교정"""
    spacing = pykospacing.Spacing()
    df["sentence_1"] = df["sentence_1"].map(spacing)
    df["sentence_2"] = df["sentence_2"].map(spacing)
    return df

In [6]:
def swap(df):
    """sentence 1과 2를 교환한 데이터 추가"""
    df_swaped = df.rename(
        columns={"sentence_1": "sentence_2", "sentence_2": "sentence_1"}
    )
    return pd.concat([df, df_swaped])


def swap_over_one_label(df):
    """sentence 1과 2를 교환한 데이터 추가"""
    df_swaped = df.rename(
        columns={"sentence_1": "sentence_2", "sentence_2": "sentence_1"}
    )
    df_filtered = df_swaped[df_swaped["label"] >= 1]

    return pd.concat([df, df_filtered])

In [7]:
from eda import eda

# 커스텀된 EDA 함수 말고 원본 패키지 사용하고 싶다면 https://github.com/toriving/KoEDA
def apply_eda(df, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, num_aug=2):
    """
    EDA 적용 함수 \n 
    주의: 한글 제외한 나머지 문자 제거됨 \n
    alpha_sr: 특정 단어를 유의어로 교체할 확률 \n
    alpha_ri: 임의의 단어를 삽입할 확률 \n
    alpha_rs: 문장 내 임의의 두 단어의 위치를 바꿀 확률 \n
    num_aug: 데이터 증강하는 개수 \n
    """
    def _conditional_eda(row, column_name):
        if row["label"] >= 0:  
            return eda.EDA(
                row[column_name], alpha_sr, alpha_ri, alpha_rs, num_aug)
        else:
            return [row[column_name]]

    def _replace_person_token(df):
        """Speicial 토큰 처리: <PERSON> -> 궯궯궯"""
        df["sentence_1"] = df["sentence_1"].str.replace("<PERSON>", "궯궯궯")
        df["sentence_2"] = df["sentence_2"].str.replace("<PERSON>", "궯궯궯")
        return df

    def _recover_person_token(df):
        """Speicial 토큰 처리: 궯궯궯 -> <PERSON>"""
        df["sentence_1"] = df["sentence_1"].str.replace("궯궯궯", "<PERSON>")
        df["sentence_2"] = df["sentence_2"].str.replace("궯궯궯", "<PERSON>")
        return df

    df = _replace_person_token(df)
    df["sentence_1"] = df.apply(lambda row: _conditional_eda(row, "sentence_1"), axis=1)
    df = df.explode("sentence_1").reset_index(drop=True)
    df["sentence_2"] = df.apply(lambda row: _conditional_eda(row, "sentence_2"), axis=1)
    df = df.explode("sentence_2").reset_index(drop=True)
    df = _recover_person_token(df)
    
    return df

In [8]:
# 전처리 및 증강 적용한 csv 파일 생성 (원하는 함수를 선택하여 사용)
def make(df, df_name):
    df = replace_special_letters(df)
    df = spell_check(df)
    df = spacing_text(df)
    df = swap(df)
    df = apply_eda(df)
    df = df.drop_duplicates()
    df.to_csv(f"./{df_name}.csv")
    return df

In [9]:
def make_v1(df, df_name):
    df = replace_special_letters(df)
    df = spell_check(df)
    df = df.drop_duplicates()
    df.to_csv(f"../data/{df_name}.csv")
    return df


def make_v1_02(df, df_name):
    df = spell_check(df)
    df = df.drop_duplicates()
    df.to_csv(f"../data/{df_name}.csv")
    return df


def make_v2(df, df_name):
    df = replace_special_letters(df)
    df = spell_check(df)
    df = swap(df)
    df = df.drop_duplicates()
    df.to_csv(f"../data/{df_name}.csv")
    return df


def make_v2_02(df, df_name):
    df = replace_special_letters(df)
    df = spell_check(df)
    df = swap_over_one_label(df)
    df = df.drop_duplicates()
    df.to_csv(f"../data/{df_name}.csv")
    return df


def make_v2_03(df, df_name):
    df = spell_check(df)
    df = swap_over_one_label(df)
    df = df.drop_duplicates()
    df.to_csv(f"../data/{df_name}.csv")
    return df


def make_v3(df, df_name):
    df = replace_special_letters(df)
    df = spell_check(df)
    df = swap_over_one_label(df)
    df = apply_eda(df, alpha_sr=0.3, alpha_ri=0.3, alpha_rs=0.3)
    df = df.drop_duplicates()
    df.to_csv(f"../data/{df_name}.csv")
    return df

In [10]:
train = pd.read_csv("../data/raw/train.csv", encoding="UTF-8")
dev = pd.read_csv("../data/raw/dev.csv", encoding="UTF-8")

train_v1 = make_v1_02(train, "train_v1_02")
dev_v1 = make_v1_02(dev, "dev_v1_02")

print_label_count(train_v1)
print_label_count(dev_v1)

ParseError: not well-formed (invalid token): line 1, column 14 (<string>)